In [8]:
import os
import shutil
import subprocess

import mrcfile
import numpy as np
from scipy.ndimage import zoom
import json
import os
import shutil
import numpy as np

import pandas as pd

from MRC import MRC

In [2]:
def download_map_model(emdb, pdb, resolution, directory):
    emdb_id = emdb.split("-")[1]
    directory1 = f"{directory}/{emdb}_re_{resolution}"
    os.makedirs(directory1, exist_ok=True)
    emdb_fetch_link = f"https://ftp.ebi.ac.uk/pub/databases/emdb/structures/{emdb}/map/emd_{emdb_id}.map.gz"
    pdb_fetch_link = f"http://files.rcsb.org/download/{pdb}.cif"
    if os.path.exists(f"{directory1}/emd_{emdb_id}.map"):
        pass
    else:
        subprocess.run(["wget", "-P", directory1, emdb_fetch_link])
        subprocess.run(["gzip", "-d", f"{directory1}/emd_{emdb_id}.map.gz"])
    if os.path.exists(f"{directory1}/{pdb}.cif"):
        pass
    else:
        subprocess.run(["wget", "-P", directory1, pdb_fetch_link])
        print(f"=> {emdb} and pdb-{pdb} files are downloaded.")

In [20]:
def map_normalizing(map_path, output_map):
    with mrcfile.mmap(map_path) as mrc:
        # Load map data
        map_data = np.array(mrc.data, dtype=np.float32)

        # Resample map to 1.0A*1.0A*1.0A grid size
        zoom_factors = [mrc.voxel_size.z, mrc.voxel_size.y, mrc.voxel_size.x]
        map_data = zoom(map_data, zoom_factors)

        # Normalize map values to the range (0.0, 1.0)
        data_99_9 = np.percentile(map_data, 99.9)
        if data_99_9 == 0.:
            print('data_99_9 == 0!!')
            raise ValueError('99.9th percentile of map data is zero')
        map_data /= data_99_9
        map_data = np.clip(map_data, 0., 1.)

        # Calculate the start of z,y,x axis
        if mrc.header.origin.z!=0 or mrc.header.origin.y!=0 or mrc.header.origin.x!=0:
            print("The origin is not (0, 0, 0)")
            exit()
        nzstart = mrc.header.nzstart * mrc.voxel_size.z
        nystart = mrc.header.nystart * mrc.voxel_size.y
        nxstart = mrc.header.nystart * mrc.voxel_size.x
        mrc.print_header()


    if os.path.exists(output_map):
        os.remove(output_map)
    print(f"=> Writing new map to {output_map}")
    shutil.copyfile(map_path, output_map)
    with mrcfile.open(output_map, mode='r+') as mrco:
        mrco.set_data(map_data)
        mrco.header.mz = map_data.shape[0]
        mrco.header.ispg = 1  #401
        mrco.header.nzstart = nzstart
        mrco.header.nystart = nystart
        mrco.header.nxstart = nxstart
        mrco.print_header()
        print("=> New map written successfully.")


In [3]:
def map_output(input_map, map_data, output_map, is_model=False):
    if os.path.exists(output_map):
        os.remove(output_map)

    print(f"=> Writing new map to {output_map}")
    shutil.copyfile(input_map, output_map)
    with mrcfile.open(output_map, mode='r+') as mrc:
        if is_model:
            map_data = map_data.astype(np.int8)
        else:
            map_data = map_data.astype(np.float32)

        mrc.set_data(map_data)
        mrc.header.mz = map_data.shape[0]
        mrc.header.ispg = 1  #401
        mrc.print_header()
        print("=> New map written successfully.")

In [9]:
DOWNLOAD = False
NORMALIZATION = True
MAP2NPY = False

In [10]:
MODEL_PARTS = ['secondary_strctures', 'key_atoms', 'residue_types']

MAIN_HOME_PATH = '/home/qiboxu/Database/U_NET/EMDB_PDB_for_U_Net'
PATH_SETTINGS = {
    "Filtered_Dateset": [
        os.path.join(MAIN_HOME_PATH, 'Filtered_Dateset', 'Raw'),
        os.path.join(MAIN_HOME_PATH, 'Filtered_Dateset', 'Training'),
        os.path.join(MAIN_HOME_PATH, 'Filtered_Dateset', 'Raw',
                     'final-20240212.csv'),
    ],
}

PATH_KEYS = "Filtered_Dateset"
DATA_PATH, HOME_PATH, csv_path = PATH_SETTINGS[PATH_KEYS]
temp_sample_path = os.path.join(HOME_PATH, "ready_to_train_and_val")
os.makedirs(temp_sample_path, exist_ok=True)


In [11]:
# Read map list and generate raw_map and model downloading paths
df = pd.read_csv(csv_path)
emdbs, pdbs = df["emdb_id"], df["fitted_pdbs"]
resolutions = df["resolution"].astype(str)
emdb_ids = [emdb.split("-")[1] for emdb in emdbs]
folders = [
    f"{emdb}_re_{resolution}" for emdb, resolution in zip(emdbs, resolutions)
]
raw_maps = [f"emd_{emdb_id}.map" for emdb_id in emdb_ids]
models = [f"{pdb}.cif" for pdb in pdbs]
raw_map_paths = [
    f"{DATA_PATH}/{folder}/{raw_map}"
    for folder, raw_map in zip(folders, raw_maps)
]
model_paths = [
    f"{DATA_PATH}/{folder}/{model}" for folder, model in zip(folders, models)
]

In [26]:
for i, raw_map_path in enumerate(raw_map_paths):
    if i == 114:
        print(raw_map_path)
        map_path = f"{raw_map_path.split('.map')[0]}_normalized.mrc"
    
        if NORMALIZATION and not os.path.exists(map_path):
            map_normalizing(raw_map_path, map_path)


/home/qiboxu/Database/U_NET/EMDB_PDB_for_U_Net/Filtered_Dateset/Raw/EMD-25568_re_3.9/emd_25568.map
nx              : 128
ny              : 137
nz              : 122
mode            : 2
nxstart         : 55
nystart         : 52
nzstart         : 60
mx              : 122
my              : 137
mz              : 128
cella           : (129.198, 145.08301, 135.552)
cellb           : (90., 90., 90.)
mapc            : 3
mapr            : 2
maps            : 1
dmin            : -8.97205924987793
dmax            : 17.262170791625977
dmean           : -8.171292635583693e-12
ispg            : 1
nsymbt          : 0
extra1          : b'\x00\x00\x00\x00\x00\x00\x00\x00'
exttyp          : b''
nversion        : 20140
extra2          : b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x